## Use a Local LLM model 

In [4]:
# Llama2 7B parameter model
# wget https://huggingface.co/localmodels/Llama-2-7B-Chat-ggml/resolve/main/llama-2-7b-chat.ggmlv3.q4_K_S.bin
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [5]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(model_path="/Users/snemati/Documents/Git_Repo/h2ogpt/llama-2-7b-chat.ggmlv3.q4_K_S.bin", n_ctx=2048)

llama.cpp: loading model from /Users/snemati/Documents/Git_Repo/h2ogpt/llama-2-7b-chat.ggmlv3.q4_K_S.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 14 (mostly Q4_K - Small)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 3647.96 MB (+ 1024.00 MB per state)
llama_new_context_

In [8]:
llm("What is Llama.cpp?") # What do you think about the quality of this response? We will get back this later.

Llama.generate: prefix-match hit

llama_print_timings:        load time =  9313.78 ms
llama_print_timings:      sample time =   382.23 ms /   220 runs   (    1.74 ms per token,   575.56 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 46892.12 ms /   220 runs   (  213.15 ms per token,     4.69 tokens per second)
llama_print_timings:       total time = 48065.69 ms


'\nLlama.cpp is a C++ library for building and using linters, specifically the Llama language. It provides an easy-to-use API for defining and executing linters, as well as some utilities for working with linting rules.\n\nThe main components of Llama.cpp are:\n\n1. A set of standard linters that can be used to check code against a set of rules. These include checks for things like coding style, naming conventions, and best practices.\n2. A mechanism for defining custom linting rules. This allows developers to create their own checks and enforce their own coding standards.\n3. Utilities for working with linting rules, such as parsing and validating them.\n\nLlama.cpp is designed to be easy to use and flexible, allowing developers to easily integrate it into their development workflows. It can be used in combination with other tools, such as linters and code editors, to provide a complete linting solution.'

In [10]:
llm("What is SDoH?") # What do you think about the quality of this response? 

Llama.generate: prefix-match hit

llama_print_timings:        load time =  9313.78 ms
llama_print_timings:      sample time =   205.79 ms /   122 runs   (    1.69 ms per token,   592.84 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 23308.80 ms /   122 runs   (  191.06 ms per token,     5.23 tokens per second)
llama_print_timings:       total time = 23934.24 ms


'\n Why is it important for healthcare providers to understand and address Social Determinants of Health (SDoH)?\n What are the key drivers of social determinants of health?\n How can healthcare providers use data to better understand SDoH and improve patient outcomes?\n What are some potential challenges or limitations of using data to address SDoH?\nWhat is the role of policy makers in addressing SDoH through legislation or regulatory actions?\nHow can community-based organizations play a critical role in improving social determinants of health?'

### Let's perform named entity recognition

In [110]:
template = """
              Detect named entities in following text delimited by triple backquotes.
              Return your response in json format "named entity","type".
              Return all entities
              ```{text}```
              json format file:
           """
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [111]:
clinical_note = """A 28-year-old previously healthy adult patient presented with tachycardia, fever, and mental confusion. The symptoms started after a cut to his leg while gardening."""
answer = llm_chain.run(clinical_note)
print(answer)

Llama.generate: prefix-match hit


 {
                "named entity", "type"
                "patient", "person"
                "symptoms", "medical_sign"
                "cut", "medical_procedure"
            }

        Answer:
        Sure, I can help you with that! Here's the response from the trained model in JSON format:
        {
            "named entity": [
                "patient",
                "symptoms",
                "cut"
            ],
            "type": [
                "person",
                "medical_sign",
                "medical_procedure"
            ]
        }

Explanation:
The trained model has detected the following named entities in the given text:

* "patient": A person who is the subject of the medical condition. (Type: "person")
* "symptoms": The medical signs or conditions exhibited by a patient. (Type: "medical_sign")
* "cut": A medical procedure that involves cutting or removing a part of the body. (Type: "medical_procedure")

These named entities are then mapped to their corre


llama_print_timings:        load time =  7200.78 ms
llama_print_timings:      sample time =   276.03 ms /   256 runs   (    1.08 ms per token,   927.43 tokens per second)
llama_print_timings: prompt eval time =  8366.51 ms /    92 tokens (   90.94 ms per token,    11.00 tokens per second)
llama_print_timings:        eval time = 33991.92 ms /   255 runs   (  133.30 ms per token,     7.50 tokens per second)
llama_print_timings:       total time = 43292.54 ms


### Okay, now let's look at an example of differential diagnosis

In [112]:
template = """
              You are an ER doctor.
              Return the top three differentials in following text delimited by triple backquotes.
              ```{text}```
              Stay concise.
           """
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [113]:
print(llm_chain.run(clinical_note))

Llama.generate: prefix-match hit


 ```
            1. Bacterial sepsis
            2. Meningitis
            3. Encephalitis
            ```
     



llama_print_timings:        load time =  7200.78 ms
llama_print_timings:      sample time =    35.11 ms /    32 runs   (    1.10 ms per token,   911.53 tokens per second)
llama_print_timings: prompt eval time =  7469.38 ms /    80 tokens (   93.37 ms per token,    10.71 tokens per second)
llama_print_timings:        eval time =  3857.90 ms /    32 runs   (  120.56 ms per token,     8.29 tokens per second)
llama_print_timings:       total time = 11454.35 ms


### Now let's try RAG with our local LLM

In [126]:
## Extracting and organizing history of present illness into predetermined categories
from langchain.embeddings import LlamaCppEmbeddings
# Make sure the model path is correct for your system!
llama = LlamaCppEmbeddings(model_path="/Users/snemati/Documents/Git_Repo/h2ogpt/llama-2-7b-chat.ggmlv3.q4_K_S.bin")

vectorstore = FAISS.from_texts([clinical_note], embedding=llama)
retriever = vectorstore.as_retriever()

template = """Answer the Question by addressing the following 10 categories:
              1. General
              2. Skin
              3. HEENT
              4. Pulmonary
              5. Cardiovascular
              6. Gastrointestinal
              7. Genitourinary
              8. Musculoskeletal
              9. Neurologic
              10. Psychiatric
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = llm

llama.cpp: loading model from /Users/snemati/Documents/Git_Repo/h2ogpt/llama-2-7b-chat.ggmlv3.q4_K_S.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 14 (mostly Q4_K - Small)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 3647.96 MB (+  512.00 MB per state)
llama_new_context_w

In [127]:
chain = ( {"context": retriever, "question": RunnablePassthrough()} | prompt  | model | StrOutputParser() )
chain.invoke("What is the history of present illness?")



llama_print_timings:        load time =  4702.62 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =  1405.86 ms /    10 tokens (  140.59 ms per token,     7.11 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  1412.03 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  7200.78 ms
llama_print_timings:      sample time =   402.99 ms /   256 runs   (    1.57 ms per token,   635.26 tokens per second)
llama_print_timings: prompt eval time = 19221.57 ms /   147 tokens (  130.76 ms per token,     7.65 tokens per second)
llama_print_timings:        eval time = 65259.17 ms /   255 runs   (  255.92 ms per token,     3.91 tokens per second)
llama_print_timings:       total time = 86322.09 ms


"\nAnswer:\n\n1. General: The patient presents with tachycardia (rapid heart rate), fever (elevated body temperature), and mental confusion, which are indicative of a systemic inflammatory response. These symptoms suggest that the patient's body is responding to an infection or injury, rather than a localized cut to the leg.\n2. Skin: There is no mention of any skin changes or lesions related to the cut or the infection.\n3. HEENT: There is no information about any changes in the patient's breathing, nasal discharge, or eyes.\n4. Pulmonary: The patient's symptoms do not suggest any pulmonary involvement, such as cough or shortness of breath.\n5. Cardiovascular: The patient's tachycardia (rapid heart rate) is a cardiovascular symptom that may indicate an infection or inflammation affecting the cardiovascular system.\n6. Gastrointestinal: There is no information about any gastrointestinal symptoms, such as abdominal pain or diarrhea"

In [134]:
# How about a physical exam with some vitals and impressions?
vectorstore = FAISS.from_texts([clinical_note,"Temperature=99, Pulse=110, BP=100/60, Respiration=24, SpO2=92%", 'body weight = 85 kg', "redness, warmth, tenderness, and swelling of the skin"], 
                               embedding=llama)
retriever = vectorstore.as_retriever()

template = """Answer the Question by addressing the following 11 categories:
            1. Vital signs
            	- Temperature
            	- Pulse
            	- BP
            	- Respirations 
            	- SpO2
            2. General appearance
            3. Skin
            4. HEENT
            5. Pulmonary
            6. Cardiovascular
            7. Gastrointestinal
            8. Genitourinary
            9. Musculoskeletal
            10. Neurologic
            11. Mental status
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = llm


llama_print_timings:        load time =  4702.62 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =  5746.72 ms /    43 tokens (  133.64 ms per token,     7.48 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  5763.96 ms

llama_print_timings:        load time =  4702.62 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time = 15309.41 ms /    39 tokens (  392.55 ms per token,     2.55 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time = 15335.04 ms

llama_print_timings:        load time =  4702.62 ms
llama_print_timings:   

In [135]:
chain = ( {"context": retriever, "question": RunnablePassthrough()} | prompt  | model | StrOutputParser() )
chain.invoke("What is the physical examination results?")


llama_print_timings:        load time =  4702.62 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =  3068.83 ms /     8 tokens (  383.60 ms per token,     2.61 tokens per second)
llama_print_timings:        eval time =   359.79 ms /     1 runs   (  359.79 ms per token,     2.78 tokens per second)
llama_print_timings:       total time =  3441.40 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  7200.78 ms
llama_print_timings:      sample time =   505.11 ms /   256 runs   (    1.97 ms per token,   506.82 tokens per second)
llama_print_timings: prompt eval time = 80939.63 ms /   261 tokens (  310.11 ms per token,     3.22 tokens per second)
llama_print_timings:        eval time = 74158.55 ms /   255 runs   (  290.82 ms per token,     3.44 tokens per second)
llama_print_timings:       total time = 157027.58 ms


'\nSolution: Based on the given information, the physical examination results are as follows:\n\n1. Vital signs:\n\t* Temperature: 99°F (37.2°C)\n\t* Pulse: 110 beats per minute (bpm)\n\t* Blood pressure (BP): 100/60 mmHg\n\t* Respirations: 24 breaths per minute (bpm)\n\t* SpO2: 92%\n2. General appearance: There is redness, warmth, tenderness, and swelling of the skin.\n3. Skin: The skin appears red, warm, and tender to the touch.\n4. HEENT: None specified in the given information.\n5. Pulmonary: None specified in the given information.\n6. Cardiovascular: Tachycardia (fast heart rate) is present.\n7. Gastrointestinal: None specified in the given information.\n8. Genitourinary: None specified in the given information.\n9. Musculoskeletal: None specified in the given information.\n'

## Let's try this with ChatGPT!!!

In [ ]:
import openai
from openai import ChatCompletion,Completion
import os
openai.organization = "org-mGi0K4AmKu41HxKowsVRjoNh"
openai.api_key = os.getenv("OPENAI_API_KEY")
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

In [120]:
# physical exam with some vitals and impressions
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))
vectorstore = FAISS.from_texts([clinical_note,"Temperature=99, Pulse=110, BP=100/60, Respiration=24, SpO2=92%", "redness, warmth, tenderness, and swelling of the skin"], 
                               embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """Answer the Question by addressing the following 11 categories:
            1. Vital signs
            	- Temperature
            	- Pulse
            	- BP
            	- Respirations 
            	- SpO2
            2. General appearance
            3. Skin
            4. HEENT
            5. Pulmonary
            6. Cardiovascular
            7. Gastrointestinal
            8. Genitourinary
            9. Musculoskeletal
            10. Neurologic
            11. Mental status
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

In [121]:
chain = ( {"context": retriever, "question": RunnablePassthrough()} | prompt  | model | StrOutputParser() )
chain.invoke("What is the physical examination results?")

"Based on the provided information, the physical examination results are as follows:\n\n1. Vital signs:\n   - Temperature: 99°F\n   - Pulse: 110 beats per minute\n   - Blood Pressure: 100/60 mmHg\n   - Respirations: 24 breaths per minute\n   - SpO2: 92%\n\n2. General appearance: Not mentioned in the provided information.\n\n3. Skin: The patient has redness, warmth, tenderness, and swelling of the skin, which may indicate inflammation or infection.\n\n4. HEENT (Head, Eyes, Ears, Nose, and Throat): Not mentioned in the provided information.\n\n5. Pulmonary: Not mentioned in the provided information, but the patient's SpO2 level of 92% suggests a potential issue with oxygenation.\n\n6. Cardiovascular: Not mentioned in the provided information, except for tachycardia (elevated heart rate) of 110 beats per minute.\n\n7. Gastrointestinal: Not mentioned in the provided information.\n\n8. Genitourinary: Not mentioned in the provided information.\n\n9. Musculoskeletal: The patient has a cut to 

### The H2OGPT

In [ ]:
# https://github.com/h2oai/h2ogpt/blob/main/docs/README_MACOS.md --> h2oGPT personal LLM
# cd /Users/snemati/Documents/Git_Repo/h2ogpt
# conda activate h2ogpt
# python generate.py --base_model='llama' --prompt_type=llama2 --score_model=None --langchain_mode='UserData' --user_path=user_path
